In [2]:
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en", disable=['ner', 'parser'])
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [4]:
with open('abstracts.txt', 'r') as abs:
    data = abs.read().split('\n')


data_words = list(sent_to_words(data))
print(data_words[0])

['current', 'machine', 'learning', 'algorithms', 'can', 'be', 'easily', 'fooled', 'by', 'adversarial', 'examples', 'one', 'possible', 'solution', 'path', 'is', 'to', 'make', 'models', 'that', 'use', 'confidence', 'thresholding', 'to', 'avoid', 'making', 'mistakes', 'such', 'models', 'refuse', 'to', 'make', 'prediction', 'when', 'they', 'are', 'not', 'confident', 'of', 'their', 'answer', 'we', 'propose', 'to', 'evaluate', 'such', 'models', 'in', 'terms', 'of', 'tradeoff', 'curves', 'with', 'the', 'goal', 'of', 'high', 'success', 'rate', 'on', 'clean', 'examples', 'and', 'low', 'failure', 'rate', 'on', 'adversarial', 'examples', 'existing', 'untargeted', 'attacks', 'developed', 'for', 'models', 'that', 'do', 'not', 'use', 'confidence', 'thresholding', 'tend', 'to', 'underestimate', 'such', 'models', 'vulnerability', 'we', 'propose', 'the', 'maxconfidence', 'family', 'of', 'attacks', 'which', 'are', 'optimal', 'in', 'variety', 'of', 'theoretical', 'settings', 'including', 'one', 'realisti

In [20]:
len(data_words)

300

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=2) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words])

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
print(data_words_nostops[0])
#
# # Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_bigrams(data_words_nostops)
print(data_words_trigrams[0])

/home/tuor369/bin/anaconda3/envs/py3.6/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['current', 'machine', 'learning', 'algorithms', 'easily', 'fooled', 'adversarial', 'examples', 'one', 'possible', 'solution', 'path', 'make', 'models', 'confidence', 'thresholding', 'avoid', 'making', 'mistakes', 'models', 'refuse', 'make', 'prediction', 'confident', 'answer', 'propose', 'evaluate', 'models', 'terms', 'tradeoff', 'curves', 'goal', 'high', 'success', 'rate', 'clean', 'examples', 'low', 'failure', 'rate', 'adversarial', 'examples', 'existing', 'untargeted', 'attacks', 'developed', 'models', 'confidence', 'thresholding', 'tend', 'underestimate', 'models', 'vulnerability', 'propose', 'maxconfidence', 'family', 'attacks', 'optimal', 'variety', 'theoretical', 'settings', 'including', 'one', 'realistic', 'setting', 'attacks', 'linear', 'models', 'experiments', 'show', 'attack', 'attains', 'good', 'results', 'practice', 'show', 'simple', 'defenses', 'able', 'perform', 'well', 'mnist', 'cifar', 'contributing', 'previous', 'calls', 'mnist', 'retired', 'benchmarking', 'dataset',

In [11]:
# # Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
#
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

# Human readable format of corpus (term-frequency)
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

[['current', 'machine_learn', 'algorithm', 'easily', 'fool', 'adversarial_exampl', 'possible', 'solution', 'path', 'make', 'model', 'confidence', 'threshold', 'avoid', 'make', 'mistake', 'model', 'refuse', 'make', 'prediction', 'confident', 'answer', 'propose', 'evaluate', 'model', 'term', 'tradeoff', 'curve', 'goal', 'high_success', 'rate', 'clean_example', 'low', 'failure', 'rate', 'adversarial_exampl', 'exist', 'untargeted', 'attack', 'develop', 'model', 'confidence', 'thresholding', 'tend', 'underestimate', 'model', 'vulnerability', 'propose', 'maxconfidence', 'family', 'attack', 'optimal', 'variety', 'theoretical', 'setting', 'include', 'realistic', 'set', 'attack', 'linear', 'model', 'experiments_show', 'attack', 'attain', 'good', 'result', 'practice', 'show', 'simple', 'defense', 'able', 'perform_well', 'mnist_cifar', 'contribute', 'previous', 'call', 'mnist', 'retire', 'benchmark', 'dataset', 'adversarial', 'robustness', 'research', 'release', 'code', 'evaluation', 'part', 'cle

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"model" + 0.023*"datum" + 0.015*"machine_learn" + '
  '0.013*"poisoning_attack" + 0.012*"access" + 0.011*"algorithm" + '
  '0.010*"training_data" + 0.009*"system" + 0.009*"privacy" + 0.009*"policy"'),
 (1,
  '0.023*"backdoor" + 0.013*"system" + 0.009*"audio" + 0.008*"trigger" + '
  '0.008*"voice_command" + 0.006*"exploit" + 0.006*"privacy" + 0.006*"build" + '
  '0.006*"channel" + 0.006*"human"'),
 (2,
  '0.031*"attack" + 0.028*"model" + 0.017*"adversarial_example" + '
  '0.015*"adversarial_exampl" + 0.009*"show" + 0.008*"detection" + '
  '0.008*"different" + 0.007*"propose" + 0.007*"approach" + 0.007*"example"'),
 (3,
  '0.014*"system" + 0.014*"dnn" + 0.012*"security" + 0.011*"model" + '
  '0.008*"attack" + 0.008*"face_recognition" + 0.007*"deep_learn" + '
  '0.007*"feature" + 0.007*"ml" + 0.006*"human"'),
 (4,
  '0.021*"object" + 0.019*"method" + 0.016*"problem" + 0.015*"cnn" + '
  '0.014*"lead" + 0.012*"agent" + 0.012*"segmentation" + 0.011*"image" + '
  '0.011*"exhaust

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.4581499371564215


In [18]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.41470010757513753


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/tuor369/bin/anaconda3/envs/py3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.189869 -0.029717       1        1  34.897873
8      0.176076  0.061074       2        1  20.375727
3      0.100170  0.112589       3        1  11.010261
2      0.136281 -0.017956       4        1  10.105816
1     -0.225489  0.075919       5        1   5.108722
9     -0.030482 -0.099693       6        1   4.535292
7     -0.017532 -0.237074       7        1   4.409786
0     -0.033050  0.126915       8        1   4.034263
4     -0.159861 -0.052592       9        1   2.974166
5     -0.135982  0.060534      10        1   2.548094, topic_info=     Category        Freq                      Term       Total  loglift  \
term                                                                       
42    Default  440.000000                     model  440.000000  30.0000   
208   Default  121.000000                classifier  121.000000  29.0000   
1     Default  269.000000               adversarial  269.000000  28.0000   
405   Default  174.000000                    method  174.000000  27.0000   
110   Default  113.000000              perturbation  113.000000  26.0000   
140   Default  113.000000                     datum  113.000000  25.0000   
191   Default  170.000000                    system  170.000000  24.0000   
363   Default  183.000000                     image  183.000000  23.0000   
5     Default  511.000000                    attack  511.000000  22.0000   
315   Default   96.000000                       dnn   96.000000  21.0000   
304   Default   73.000000  adversarial_perturbation   73.000000  20.0000   
77    Default  138.000000       adversarial_example  138.000000  19.0000   
36    Default  135.000000             machine_learn  135.000000  18.0000   
175   Default   61.000000                   problem   61.000000  17.0000   
545   Default   61.000000              distribution   61.000000  16.0000   
60    Default   74.000000                robustness   74.000000  15.0000   
2258  Default   32.000000                  backdoor   32.000000  14.0000   
664   Default   99.000000            neural_network   99.000000  13.0000   
3     Default  123.000000                 algorithm  123.000000  12.0000   
2     Default   90.000000        adversarial_exampl   90.000000  11.0000   
161   Default   39.000000                      lead   39.000000  10.0000   
1760  Default   27.000000                  detector   27.000000   9.0000   
58    Default   88.000000                    result   88.000000   8.0000   
490   Default   48.000000             training_data   48.000000   7.0000   
130   Default  110.000000                 adversary  110.000000   6.0000   
186   Default   71.000000                  security   71.000000   5.0000   
418   Default   78.000000                     state   78.000000   4.0000   
18    Default   74.000000                   dataset   74.000000   3.0000   
1399  Default   34.000000                    access   34.000000   2.0000   
24    Default   48.000000                     exist   48.000000   1.0000   
...       ...         ...                       ...         ...      ...   
2800  Topic10    2.336064                 seriously    3.077950   3.3940   
637   Topic10    2.166189                  test_cas    2.907319   3.3756   
2110  Topic10    2.166180                   fuzzing    2.907319   3.3756   
2356  Topic10    2.098175                      apis    2.839366   3.3673   
2679  Topic10    4.828344                 criterion    6.620314   3.3542   
627   Topic10    4.854188                    expert    6.713805   3.3455   
631   Topic10    1.927831                perception    2.668933   3.3445   
3051  Topic10    1.859865                   sensory    2.600962   3.3344   
1608  Topic10    3.575325                       bug    5.053252   3.3238   
3377  Topic10    1.737372                  mutation    2.478500   3.3145   
3378  Topic10    1.737372                